In [1]:
# encoding = utf8
'''
    @Author: King
    @Date: 2019.05.16
    @Purpose: Attention-Based-BiLSTM-relation-extraction
    @Introduction:  Attention-Based-BiLSTM-relation-extraction
    @Datasets: Chinese relation extration datasets
    @Link : 
    @Reference : https://github.com/SeoSangwoo/Attention-Based-BiLSTM-relation-extraction
'''

'\n    @Author: King\n    @Date: 2019.05.16\n    @Purpose: Attention-Based-BiLSTM-relation-extraction\n    @Introduction:  Attention-Based-BiLSTM-relation-extraction\n    @Datasets: Chinese relation extration datasets\n    @Link : \n    @Reference : https://github.com/SeoSangwoo/Attention-Based-BiLSTM-relation-extraction\n'

## Attention-Based Bidirectional Long Short-Term Memory Networks for Relation Classification

Tensorflow Implementation of Deep Learning Approach for Relation Extraction Challenge(SemEval-2010 Task #8: Multi-Way Classification of Semantic Relations Between Pairs of Nominals) via Attention-based BiLSTM.

Original paper [Attention-Based Bidirectional Long Short-Term Memory Networks for Relation Classification](http://anthology.aclweb.org/P16-2034) 

![Attention-Based-BiLSTM-relation-extraction](img/Attention-Based-BiLSTM-relation-extraction.png)


### Requrements

* Python (>=3.5)

* TensorFlow (>=r1.0)

* scikit-learn (>=0.18)


### 1、Settings Class

In [5]:
class Settings(object):
    def __init__(self):
        
        '''
            Data loading params 
        '''
        ## Path of train data
        self.train_path = "SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT"
        self.train_path_cn = "E:/pythonWp/nlp/relation_extraction/Information-Extraction-Chinese_suss/RE_BGRU_2ATT/origin_data/test_char.txt"
        # Path of test data
        self.test_path = "SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT"
        # Path of relation2id data
        self.relation2id_path = "E:/pythonWp/nlp/relation_extraction/Information-Extraction-Chinese_suss/RE_BGRU_2ATT/origin_data/relation2id.txt"
        # Max sentence length in data
        self.max_sentence_length = 90
        # Percentage of the training data to use for validation
        self.dev_sample_percentage = 0.1
        
        '''
            Model Hyper-parameters 
        '''
        '''
            1、Embeddings
        '''
        # Path of pre-trained word embeddings 
        self.embedding_path = "E:/pythonWp/nlp/relation_extraction/Information-Extraction-Chinese_suss/RE_BGRU_2ATT/origin_data/vec_char.txt"
        # Dimensionality of word embedding (default: 100)
        self.embedding_dim = 100
        # Dropout keep probability of embedding layer (default: 0.7)
        self.emb_dropout_keep_prob = 0.7
        
        '''
            2、AttLSTM
        '''
        # Dimensionality of RNN hidden (default: 100)
        self.hidden_size = 100
        # Dropout keep probability of RNN (default: 0.7)
        self.rnn_dropout_keep_prob = 0.7
        
        '''
            3、Misc
        '''
        # Description for model
        self.desc = ""
        # Dropout keep probability of RNN (default: 0.7)
        self.dropout_keep_prob = 0.5
        # L2 regularization lambda (default: 1e-5)
        self.l2_reg_lambda = 1e-5
        
        '''
            4、Training parameters
        '''
        # Description for model
        self.batch_size = 10
        # Number of training epochs (Default: 100)
        self.num_epochs = 100
        # Number of iterations to display training information
        self.display_every = 10
        # Evaluate model on dev set after this many steps (default: 100)
        self.evaluate_every = 100
        # Number of checkpoints to store (default: 5)
        self.num_checkpoints = 5
        # Which learning rate to start with (Default: 1.0)
        self.learning_rate = 1.0
        # Decay rate for learning rate (Default: 0.9)
        self.decay_rate = 0.9
        
        '''
            5、Testing parameters
        '''
        # Checkpoint directory from training run
        self.checkpoint_dir = ""
        
        '''
            6、Misc Parameters
        '''
        # Allow device soft device placement
        self.allow_soft_placement = True
        # Log placement of ops on devices
        self.log_device_placement = False
        # Allow gpu memory growth
        self.gpu_allow_growth = True
        

### 2、数据处理模型

In [6]:
import numpy as np
import pandas as pd 
'''
    工具包 begin
'''
import sys
if sys.version_info[0] > 2:
    is_py3 = True
else:
    reload(sys)
    sys.setdefaultencoding("utf-8")
    is_py3 = False

def native_word(word, encoding='utf-8'):
    """如果在python2下面使用python3训练的模型，可考虑调用此函数转化一下字符编码"""
    if not is_py3:
        return word.encode(encoding)
    else:
        return word

def native_content(content):
    if not is_py3:
        return content.decode('utf-8')
    else:
        return content

def open_file(filename, mode='r'):
    """
    常用文件操作，可在python2和python3间切换.
    mode: 'r' or 'w' for read or write
    """
    if is_py3:
        return open(filename, mode, encoding='utf-8', errors='ignore')
    else:
        return open(filename, mode)

'''
    工具包 end
'''


'\n    工具包 end\n'

In [7]:
#读取 relation2id 文件数据
def load_relation2id_file_cn(filename,demo_flag = False):
    '''
    读取 data 文件数据
    :param filename:    String 文件名称包含路径
    :param demo_flag:   String True 只读取 1000 样本数据，Fasle 读取全部数据
    :return:
        relation2id:   dict    relation to id
        id2relation:   list    id to relation 
    '''
    contents_num = 0
    relation2id = {}
    id2relation = []
    with open_file(filename) as f:
        for line in f:
            try:
                data_list = line.replace("\n","").split("\t")
                relation2id[data_list[0]] = int(data_list[1])
                id2relation.append(data_list[0])
                contents_num = contents_num + 1
                if demo_flag and contents_num == 500:
                    break
            except:
                pass
    return relation2id,id2relation

In [10]:
# 读取 训练集 文件数据
def load_data_and_labels_cn(path,settings):
    relation2id,id2relation=load_relation2id_file_cn(filename=settings.relation2id_path)   
    data = []

    df = pd.read_csv(path, sep='\t',names=['e1','e2','r','s'])
    print("df:{0}".format(df.iloc[0:2]))
    ''' print("df:{0}".format(df.iloc[0:2]))
        output:
            df:    
            e1  e2    r                                                  s
            0  李烈钧  王侃  NaN              李烈钧加入同盟会:光绪三十三年（1907年），经张断、王侃介绍加入同盟会。
            1   陈尸  孔子  NaN  子服景伯把这件事告诉给孔子，并且说：“季孙氏已经被公伯寮迷惑了，我的力量能够把公伯寮杀了，把...
    '''
    max_sentence_length = 0
    sentence_list = df['s'].tolist()
    for i in range(0,len(sentence_list)):
        tokens = str(sentence_list[i])
        if max_sentence_length < len(tokens):
            max_sentence_length = len(tokens)
        sentence_list[i] = " ".join(tokens)
    print("sentence_list:{0}".format(sentence_list[0:2]))
    print("max sentence length = {}\n".format(max_sentence_length))
    print("df:{0}".format(df.iloc[0:2]))
    ''' print("sentence_list:{0}".format(sentence_list[0:2]))
        print("max sentence length = {}\n".format(max_sentence_length))
        print("df:{0}".format(df.iloc[0:2]))
        output
            sentence_list:['李 烈 钧 加 入 同 盟 会 : 光 绪 三 十 三 年 （ 1 9 0 7 年 ） ， 经 张 断 、 王 侃 介 绍 加 入 同 盟 会  。', '子 服 景 伯 把 这 件 事 告 诉 给 孔 子 ， 并 且 说 ： “ 季 孙 氏 已 经 被 公 伯 寮 迷 惑 了 ， 我 的 力 量 能 够 把 公 伯 寮 杀 了 ， 把 他 陈 尸 于 市 。 ”']
            max sentence length = 19751

            df:    e1  e2    r                                                  s
            0  李烈钧  王侃  NaN              李烈钧加入同盟会:光绪三十三年（1907年），经张断、王侃介绍加入同盟会。
            1   陈尸  孔子  NaN  子服景伯把这件事告诉给孔子，并且说：“季孙氏已经被公伯寮迷惑了，我的力量能够把公伯寮杀了，把...
    '''

    df = df.fillna('NA')            # 将省缺值用 ‘NAN’ 代替

    df['label'] = [relation2id[str(r)] for r in df['r']]
    print("df:{0}".format(df.iloc[0:2]))
    ''' print("df:{0}".format(df.iloc[0:2]))
        output:
            df:    
                    e1  e2   r                                                  s  label
            0  李烈钧  王侃  NA              李烈钧加入同盟会:光绪三十三年（1907年），经张断、王侃介绍加入同盟会。      0
            1   陈尸  孔子  NA  子服景伯把这件事告诉给孔子，并且说：“季孙氏已经被公伯寮迷惑了，我的力量能够把公伯寮杀了，把...      0
    '''

    # Text Data
    x_text = sentence_list
    print("x_text:{0}".format(x_text[0:1]))
    ''' print("x_text:{0}".format(x_text[0:1]))
        sys.exit(0)
        output:
            x_text:['李 烈 钧 加 入 同 盟 会 : 光 绪 三 十 三 年 （ 1 9 0 7 年 ） ， 经 张 断 、 王 侃 介 绍 加 入 同 盟 会 。', '子 服 景 伯 把 这 件 事 告 诉 给 孔 子 ， 并 且 说 ： “ 季 孙 氏 已 经 被 公 伯 寮 迷 惑 了 ， 我 的 力 量 能 够 把 公 伯 寮 杀 了 ， 把 他 陈 尸 于 市 。 ”', '剪 辑 ： 朱 小 勤 、 苏 鸿 文', '区 域 创 新 体 系 的 若 干 文 献 综 述 （ 陈 广 胜 许 小 忠 徐 燕 椿 ）', '在 拍 摄 间 隙 的 时 候 ， 谭 松 韵 与 郭 俊 辰 经 常 一 起 吃 辣 条 。']
    '''
    # Label Data
    y = df['label']
    labels_flat = y.values.ravel()
    labels_count = np.unique(labels_flat).shape[0]
    print("labels_flat:{0}".format(labels_flat))
    print("labels_count:{0}".format(labels_count))
    ''' print("labels_flat:{0}".format(labels_flat))
        print("labels_count:{0}".format(labels_count))
        output:
            labels_flat:[ 0  0  0 ...  0 29 29]
            labels_count:35
    '''

    # convert class labels from scalars to one-hot vectors
    # 0  => [1 0 0 0 0 ... 0 0 0 0 0]
    # 1  => [0 1 0 0 0 ... 0 0 0 0 0]
    # ...
    # 18 => [0 0 0 0 0 ... 0 0 0 0 1]
    def dense_to_one_hot(labels_dense, num_classes):
        num_labels = labels_dense.shape[0]
        index_offset = np.arange(num_labels) * num_classes
        labels_one_hot = np.zeros((num_labels, num_classes))
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
        return labels_one_hot

    labels = dense_to_one_hot(labels_flat, labels_count)
    labels = labels.astype(np.uint8)
    print("x_text:{0}".format(x_text[0:1]))
    print("labels:{0}".format(labels[0:1]))
    print("len(x_text):{0}".format(len(x_text)))
    print("len(labels):{0}".format(len(labels)))
    ''' print("x_text:{0}".format(x_text[0:1]))
        print("labels:{0}".format(labels[0:1]))
        print("len(x_text):{0}".format(len(x_text)))
        print("len(labels):{0}".format(len(labels)))
        sys.exit(0)
        output:
            x_text:['李 烈 钧 加 入 同 盟 会 : 光 绪 三 十 三 年 （ 1 9 0 7 年 ） ， 经 张 断 、 王 侃 介 绍 加 入 同 盟 会 。', '子 服 景 伯 把 这 件 事 告 诉 给 孔 子 ， 并 且 说 ： “ 季 孙 氏 已 经 被 公 伯 寮 迷 惑 了 ， 我 的 力 量 能 够 把 公 伯 寮 杀 了 ， 把 他 陈 尸 于 市 。 ”', '剪 辑 ： 朱 小 勤 、 苏 鸿 文', '区 域 创 新 体 系 的 若 干 文 献 综 述 （ 陈 广 胜 许 小 忠 徐 燕 椿 ）', '在 拍 摄 间 隙 的 时 候 ， 谭 松 韵 与 郭 俊 辰 经 常 一 起 吃 辣 条 。']
            labels:[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
             [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
             [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
             [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
             [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
            len(x_text):37637
            len(labels):37637
    '''
    return x_text, labels

In [12]:
settings =Settings()
load_data_and_labels_cn(path=settings.train_path_cn,settings=settings)
print("")

df:    e1  e2    r                                                  s
0  李烈钧  王侃  NaN              李烈钧加入同盟会:光绪三十三年（1907年），经张断、王侃介绍加入同盟会。
1   陈尸  孔子  NaN  子服景伯把这件事告诉给孔子，并且说：“季孙氏已经被公伯寮迷惑了，我的力量能够把公伯寮杀了，把...
sentence_list:['李 烈 钧 加 入 同 盟 会 : 光 绪 三 十 三 年 （ 1 9 0 7 年 ） ， 经 张 断 、 王 侃 介 绍 加 入 同 盟 会 。', '子 服 景 伯 把 这 件 事 告 诉 给 孔 子 ， 并 且 说 ： “ 季 孙 氏 已 经 被 公 伯 寮 迷 惑 了 ， 我 的 力 量 能 够 把 公 伯 寮 杀 了 ， 把 他 陈 尸 于 市 。 ”']
max sentence length = 19751

df:    e1  e2    r                                                  s
0  李烈钧  王侃  NaN              李烈钧加入同盟会:光绪三十三年（1907年），经张断、王侃介绍加入同盟会。
1   陈尸  孔子  NaN  子服景伯把这件事告诉给孔子，并且说：“季孙氏已经被公伯寮迷惑了，我的力量能够把公伯寮杀了，把...
df:    e1  e2   r                                                  s  label
0  李烈钧  王侃  NA              李烈钧加入同盟会:光绪三十三年（1907年），经张断、王侃介绍加入同盟会。      0
1   陈尸  孔子  NA  子服景伯把这件事告诉给孔子，并且说：“季孙氏已经被公伯寮迷惑了，我的力量能够把公伯寮杀了，把...      0
x_text:['李 烈 钧 加 入 同 盟 会 : 光 绪 三 十 三 年 （ 1 9 0 7 年 ） ， 经 张 断 、 王 侃 介 绍 加 入 同 盟 会 。']
labels_flat:[ 0  0  0 ...  0 29 29]
labels_count:35
x_t